# Import Modules

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from tqdm.notebook import tqdm
from collections import namedtuple
from collections import defaultdict
from pprint import pprint

# Configure display options
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
%config InlineBackend.figure_format = 'retina'

# Add project directory to sys.path
# pdir = os.path.abspath(os.path.join(os.getcwd(), '.'))
# sys.path.insert(1, pdir)
# from myutils import *

# Other module imports
import ast, math, swifter, csv, json, itertools as it, portion as P

# Set plot style
# plt.style.use('ggplot')

# Function Definitions

In [3]:
# ************************************************* unify data format *************************************************

def set_data(df, mode='pcap', tz=0):
    def nr_serv_cel(row):
        pos = row.serv_cel_pos
        if pos == 255:
            return 65535, -160, -50
        else:
            return row[f'PCI{pos}'], row[f'RSRP{pos}'], row[f'RSRQ{pos}']
    
    if mode == 'pcap':
        common_column_names = ['seq', 'rpkg', 'frame_id', 'Timestamp', 'lost', 'excl', 'latency', 'xmit_time', 'arr_time']
        
        if df.empty:
            return pd.DataFrame(columns=common_column_names)
        
        date_columns = ['Timestamp', 'xmit_time', 'arr_time']
        df[date_columns] = df[date_columns].apply(pd.to_datetime)
        df[['seq', 'rpkg', 'frame_id']] = df[['seq', 'rpkg', 'frame_id']].astype('Int32')
        df[['latency']] = df[['latency']].astype('float32')
        df[['lost', 'excl']] = df[['lost', 'excl']].astype('boolean')

    if mode in ['lte', 'nr']:
        common_column_names = [
            'Timestamp', 'type_id', 'PCI', 'RSRP', 'RSRQ', 'serv_cel_index', 'EARFCN', 'NR_ARFCN', 
            'num_cels', 'num_neigh_cels', 'serv_cel_pos', 'PCI0', 'RSRP0', 'RSRQ0',
        ]
        
        if df.empty:
            return pd.DataFrame(columns=common_column_names)
        
        if mode == 'lte':
            columns_mapping = {
                'RSRP(dBm)': 'RSRP',
                'RSRQ(dB)': 'RSRQ',
                'Serving Cell Index': 'serv_cel_index',
                'Number of Neighbor Cells': 'num_neigh_cels',
                'Number of Detected Cells': 'num_cels',
            }
            columns_order = [*common_column_names, *df.columns[df.columns.get_loc('PCI1'):].tolist()]
            
            df = df.rename(columns=columns_mapping).reindex(columns_order, axis=1)
            df['serv_cel_index'] = np.where(df['serv_cel_index'] == '(MI)Unknown', '3_SCell', df['serv_cel_index'])
            df['num_cels'] = df['num_neigh_cels'] + 1
            df['type_id'] = 'LTE_PHY'

        if mode == 'nr':
            columns_mapping = {
                'Raster ARFCN': 'NR_ARFCN',
                'Serving Cell Index': 'serv_cel_pos',
                'Num Cells': 'num_cels',
            }
            columns_order = [*common_column_names, *df.columns[df.columns.get_loc('PCI1'):].tolist()]
            
            df = df.rename(columns=columns_mapping).reindex(columns_order, axis=1)
            df[['PCI', 'RSRP', 'RSRQ']] = df.apply(nr_serv_cel, axis=1, result_type='expand')
            df['serv_cel_index'] = np.where(df['serv_cel_pos'] == 255, df['serv_cel_index'], 'PSCell')
            df['num_neigh_cels'] = np.where(df['serv_cel_pos'] == 255, df['num_cels'], df['num_cels'] - 1)
            df['type_id'] = '5G_NR_ML1'
        
        df['Timestamp'] = pd.to_datetime(df['Timestamp']) + pd.Timedelta(hours=tz)
        df[['type_id', 'serv_cel_index']] = df[['type_id', 'serv_cel_index']].astype('category')
        df[['EARFCN', 'NR_ARFCN']] = df[['EARFCN', 'NR_ARFCN']].astype('Int32')
        df[['num_cels', 'num_neigh_cels', 'serv_cel_pos']] = df[['num_cels', 'num_neigh_cels', 'serv_cel_pos']].astype('UInt8')

        for tag in df.columns:
            if tag.startswith('PCI'):
                df[tag] = df[tag].astype('Int32')
            if tag.startswith(('RSRP', 'RSRQ')):
                df[tag] = df[tag].astype('float32')

    return df

# ************************************************* Sheng-Ru Latest Version (09-25) add try except (10-27) add nr arfcn (11-13) add BSID (11-21) *************************************************

def parse_mi_ho(df, tz=8):

    # df = pd.read_csv(f)
    df["Timestamp"] = df["Timestamp"].swifter.apply(lambda x: pd.to_datetime(x) + dt.timedelta(hours=tz))
    nr_pci = 'O'
    nr_arfcn = 0
    scells = []

    def NR_OTA(idx):

        if df["type_id"].iloc[idx] == "5G_NR_RRC_OTA_Packet": return True
        else: return False
    
    def LTE_SERV_INFO(idx):

        if df["type_id"].iloc[idx] == "LTE_RRC_Serv_Cell_Info": return True
        else: return False
    

    def find_1st_after(start_idx, target, look_after=1):
        for j in range(start_idx, len(df)):
            t_ = df["Timestamp"].iloc[j]
            if NR_OTA(j) or LTE_SERV_INFO(j):
                continue
            if (t_ - t).total_seconds() > look_after:
                return None, None
            if df[target].iloc[j] not in [0,'0'] and not np.isnan(df[target].iloc[j]):
                return t_, j
        return None, None
    
    def find_1st_before(start_idx, target, look_before=1):
        for j in range(start_idx, -1, -1):
            t_ = df["Timestamp"].iloc[j]
            if NR_OTA(j) or LTE_SERV_INFO(j):
                continue
            if (t - t_).total_seconds() > look_before:
                return None, None
            if df[target].iloc[j] not in [0,'0'] and not np.isnan(df[target].iloc[j]):
                return t_, j
        return None, None
    
    def find_1st_before_with_special_value(start_idx, target, target_value, look_before=1):
        for j in range(start_idx, -1, -1):
            t_ = df["Timestamp"].iloc[j]
            if NR_OTA(j) or LTE_SERV_INFO(j):
                continue
            if (t - t_).total_seconds() > look_before:
                return None, None
            if df[target].iloc[j] in [target_value] and not np.isnan(df[target].iloc[j]):
                return t_, j
        return None, None
    
    def find_in_D_exact(targets):

        l = []
        # In l : (second, ho_type)
        for target in targets:
            for ho in D[target]:
                l.append(((t - ho.start).total_seconds(), target))

        if len(l) != 0:
            for x in l:
                if (x[0]== 0):
                    return x[1]
        
        return None
    
    def find_in_D_first_before(targets, look_before=1):

        l = []
        # In l : (second, ho_type)
        for target in targets:
            for ho in D[target]:
                try:
                    l.append(((t - ho.end).total_seconds(), target, ho))
                except:
                    pass

        if len(l) != 0:
            closest = min(filter(lambda x: x[0] > 0, l), key=lambda x: x[0])
            if 0 <= closest[0] < look_before:
                return closest[1], closest[2]
        
        return None, None
    
    HO = namedtuple('HO',['start', 'end', 'others', 'trans'], defaults=[None,None,'',''])
    
    D = {
        'Conn_Rel':[], 
        'Conn_Req':[], # Setup
        'LTE_HO': [], # LTE -> newLTE
        'MN_HO': [], # LTE + NR -> newLTE + NR
        'MN_HO_to_eNB': [], # LTE + NR -> newLTE
        'SN_setup': [], # LTE -> LTE + NR => NR setup
        'SN_Rel': [], # LTE + NR -> LTE
        'SN_HO': [], # LTE + NR -> LTE + newNR  
        'RLF_II': [],
        'RLF_III': [],
        'SCG_RLF': [],
        'Add_SCell': [],
        }

    for i in range(len(df)):

        # Pass NR RRC packet. In NSA mode, LTE RRC packet include NR packet message.
        if NR_OTA(i) or LTE_SERV_INFO(i):
            continue

        others = ''
        t = df["Timestamp"].iloc[i]

        if df["rrcConnectionRelease"].iloc[i] == 1:
            D['Conn_Rel'].append(HO(start=t))
            nr_pci = 'O'

        if df["rrcConnectionRequest"].iloc[i] == 1:
            
            # Define end of rrcConnectionRequest to be rrcConnectionReconfigurationComplete or securityModeComplete.
            a = find_1st_after(i, 'rrcConnectionReconfigurationComplete',look_after=2)[0]
            b = find_1st_after(i, 'securityModeComplete',look_after=2)[0]
            if a is None and b is None: end = None
            elif a is None and b is not None: end = b
            elif a is not None and b is None: end = a 
            else: end = a if a > b else b
            
            serv_cell, serv_freq = df["PCI"].iloc[i], int(df["Freq"].iloc[i])
            trans = f'? -> ({serv_cell}, {serv_freq})'
            D['Conn_Req'].append(HO(start=t,end=end,trans=trans))
            nr_pci = 'O'
        
        if df["lte-rrc.t304"].iloc[i] == 1:
            
            end, _ = find_1st_after(i, 'rrcConnectionReconfigurationComplete')
            serv_cell, target_cell = df["PCI"].iloc[i], int(df['lte_targetPhysCellId'].iloc[i])
            serv_freq, target_freq = int(df["Freq"].iloc[i]), int(df['dl-CarrierFreq'].iloc[i])

            if df["SCellToAddMod-r10"].iloc[i] == 1:
                n =len(str(df["SCellIndex-r10.1"].iloc[i]).split('@'))
                others += f' Set up {n} SCell.'
            else:
                scells = []
            
            if serv_freq != target_freq:
                a,b = find_1st_before(i, "rrcConnectionReestablishmentRequest", 1)
                others += " Inter frequency HO."
                if a is not None:
                    others += " Near after RLF."
                
            if df["nr-rrc.t304"].iloc[i] == 1 and df["dualConnectivityPHR: setup (1)"].iloc[i] == 1:
                
                if serv_cell == target_cell and serv_freq == target_freq:

                    a, _ = find_1st_before(i, "rrcConnectionReestablishmentRequest", 2)
                    
                    if a is not None:

                        ho_type, ho = find_in_D_first_before(['RLF_II', 'RLF_III'], 2)
                        try:
                            others += f' Near after RLF of trans: {ho.trans}.'
                        except:
                            others += f' Near after RLF.'

                    else:
                        
                        ho_type, _ = find_in_D_first_before(['MN_HO_to_eNB', 'SN_Rel'], 2)
                        if ho_type is not None:
                            others += f' Near after {ho_type}.'
                    orig_serv = (nr_pci, nr_arfcn) if nr_pci != 'O' else 'O'
                    nr_pci = int(df['nr_physCellId'].iloc[i])
                    nr_arfcn = int(df['absoluteFrequencySSB'].iloc[i])
                    trans = f'({serv_cell}, {serv_freq}) | {orig_serv} -> ({nr_pci}, {nr_arfcn})'
                    D['SN_setup'].append(HO(start=t, end=end, others=others, trans=trans))

                else:
                    
                    nr_pci = int(df['nr_physCellId'].iloc[i])
                    trans = f'({serv_cell}, {serv_freq}) -> ({target_cell}, {target_freq}) | ({nr_pci}, {nr_arfcn})'
                    D['MN_HO'].append(HO(start=t, end=end, others=others, trans=trans))

            else:
                
                if serv_cell == target_cell and serv_freq == target_freq:

                    a, b = find_1st_before(i, "scgFailureInformationNR-r15")
                    if a is not None:
                        others += " Caused by scg-failure."
                    
                    orig_serv = (nr_pci, nr_arfcn) if nr_pci != 'O' else 'O'
                    nr_pci = 'O'
                    trans = f'({serv_cell}, {serv_freq}) | {orig_serv} -> {nr_pci}'
                    D['SN_Rel'].append(HO(start=t, end=end, others=others, trans=trans))
                    
                else:

                    a, _ = find_1st_before(i,"rrcConnectionSetup",3)
                    if a is not None:
                        others += ' Near After connection setup.'
                    if nr_pci == 'O':
                        trans = f'({serv_cell}, {serv_freq}) -> ({target_cell}, {target_freq}) | {nr_pci}'
                        D['LTE_HO'].append(HO(start=t, end=end, others=others, trans=trans))
                    else:
                        orig_serv = (nr_pci, nr_arfcn) if nr_pci != 'O' else 'O'
                        nr_pci = 'O'
                        trans = f'({serv_cell}, {serv_freq}) -> ({target_cell}, {target_freq}) | {orig_serv} -> {nr_pci}'
                        D['MN_HO_to_eNB'].append(HO(start=t, end=end, others=others, trans=trans))


        if df["nr-rrc.t304"].iloc[i] == 1 and not df["dualConnectivityPHR: setup (1)"].iloc[i] == 1:

            end, _ = find_1st_after(i,'rrcConnectionReconfigurationComplete')
        
            serv_cell, serv_freq = df["PCI"].iloc[i], int(df["Freq"].iloc[i])
            orig_serv = (nr_pci, nr_arfcn) if nr_pci != 'O' else 'O'
            nr_pci = int(df['nr_physCellId'].iloc[i])
            nr_arfcn = int(df['absoluteFrequencySSB'].iloc[i])
            trans = f'({serv_cell}, {serv_freq}) | {orig_serv} -> ({nr_pci}, {nr_arfcn})'
            D['SN_HO'].append(HO(start=t,end=end,trans=trans))


        if df["rrcConnectionReestablishmentRequest"].iloc[i] == 1:

            end1, _ = find_1st_after(i, 'rrcConnectionReestablishmentComplete', look_after=1)
            b, _ = find_1st_after(i, 'rrcConnectionReestablishmentReject', look_after=1)
            end2, _ = find_1st_after(i, 'securityModeComplete',look_after=3)

            others += ' ' + df["reestablishmentCause"].iloc[i] + '.'
            scells = []

            c, _ = find_1st_before(i, 'scgFailureInformationNR-r15', 1)
            if c != None:
                others  += ' caused by scgfailure.'
                
            serv_cell, rlf_cell = df["PCI"].iloc[i], int(df['physCellId.3'].iloc[i])
            serv_freq = int(df['Freq'].iloc[i])
            
            # Type II & Type III
            if end1 is not None: 

                orig_serv = (nr_pci, nr_arfcn) if nr_pci != 'O' else 'O'
                nr_pci = 'O'
                _, idx = find_1st_before_with_special_value(i, 'PCI', rlf_cell, look_before=10)
                try:
                    rlf_freq = int(df['Freq'].iloc[idx])
                except:
                    rlf_freq = 0
                trans = f'({rlf_cell}, {rlf_freq}) -> ({serv_cell}, {serv_freq}) | {orig_serv} -> {nr_pci}'
                D['RLF_II'].append(HO(start=t,end=end1,others=others,trans=trans))

            elif b is not None and end2 is not None:
                
                orig_serv = (nr_pci, nr_arfcn) if nr_pci != 'O' else 'O'
                nr_pci = 'O'
                _, idx = find_1st_before_with_special_value(i, 'PCI', rlf_cell, look_before=10)
                try:
                    rlf_freq = int(df['Freq'].iloc[idx])
                except:
                    rlf_freq = 0

                _, idx = find_1st_after(i, "rrcConnectionRequest", 2)
                recon_cell, recon_freq = df['PCI'].iloc[idx], int(float(df['Freq'].iloc[idx]))
                
                trans = f'({rlf_cell}, {rlf_freq}) -> ({recon_cell}, {recon_freq}) | {orig_serv} -> {nr_pci}'
                D['RLF_III'].append(HO(start=t,end=end2,others=others,trans=trans))
                
            else:
                others+=' No end.'
                D['RLF_II'].append(HO(start=t,others=others))
                print('No end for RLF')

        if df["scgFailureInformationNR-r15"].iloc[i] == 1:

            others += ' ' + df["failureType-r15"].iloc[i] + '.'
            a, idx1 = find_1st_after(i, "rrcConnectionReestablishmentRequest", look_after=1)
            b, idx2 = find_1st_after(i, "lte-rrc.t304", look_after=10)

            if a is not None:

                end1, _ = find_1st_after(idx1, 'rrcConnectionReestablishmentComplete', look_after=1)
                b, _ = find_1st_after(idx1, 'rrcConnectionReestablishmentReject', look_after=1)
                end2 = find_1st_after(idx1, 'securityModeComplete',look_after=3)[0]

                others += ' Result in rrcReestablishment.'
                    
                # Type II & Type III Result
                if end1 is not None: 
                    D['SCG_RLF'].append(HO(start=t,end=end1,others=others))
                elif b is not None and end2 is not None: 
                    D['SCG_RLF'].append(HO(start=t,end=end2,others=others))
                else:
                    others += ' No end.'
                    D['SCG_RLF'].append(HO(start=t,others=others))
                    print('No end for scg failure result in rrcReestablishment.')

            elif b is not None:

                end, _ = find_1st_after(idx2, 'rrcConnectionReconfigurationComplete')
                serv_cell, target_cell = df["PCI"].iloc[idx2], df['lte_targetPhysCellId'].iloc[idx2]
                serv_freq, target_freq = int(df["Freq"].iloc[idx2]), df['dl-CarrierFreq'].iloc[idx2]
                others += ' Result in gNB release.'
                # We do not change nr_pci here. Instead, we will change it at gNB_Rel event.
                orig_serv = (nr_pci, nr_arfcn) if nr_pci != 'O' else 'O'
                trans = f'({serv_cell}, {serv_freq}) | {orig_serv} -> O'
                
                if serv_cell == target_cell and serv_freq == target_freq:
                    D['SCG_RLF'].append(HO(start=t,end=end,others=others,trans=trans))
                else:
                    others += ' Weird gNB release.'
                    print('Weird for scg failure result in gNb Release.')
                    D['SCG_RLF'].append(HO(start=t,end=end,others=others,trans=trans))                  

            else:

                print('No end for scg failure.')
                others += ' No end.'
                D['SCG_RLF'].append(HO(start=t,others=others))
        
        if df['SCellToAddMod-r10'].iloc[i] == 1 and df['physCellId-r10'].iloc[i] != 'nr or cqi report':

            others = ''
            pcis = str(df["physCellId-r10"].iloc[i]).split('@')
            freqs = str(df["dl-CarrierFreq-r10"].iloc[i]).split('@')
            orig_scells = scells
            scells = [(int(float(pci)), int(float(freq))) for pci, freq in zip(pcis, freqs)]

            others += f' Set up {len(scells)} SCell.'
            trans = f'{orig_scells} -> {scells}'

            end, _ = find_1st_after(i,'rrcConnectionReconfigurationComplete')
            
            a, _ = find_1st_before(i, "rrcConnectionReestablishmentRequest", 3)
            if a is not None:
                others += ' Near after RLF.'

            a = find_in_D_exact(['LTE_HO', 'MN_HO', 'MN_HO_to_eNB', 'SN_setup', 'SN_Rel'])
            if a is not None:
                others += f' With {a}.'

            D['Add_SCell'].append(HO(start=t,end=end,others=others, trans=trans))
    
    return D

# ************************************************* parse into readable dataframe *************************************************

def mi_parse_handover(df, tz=8, radical=True, endfill=False):
    
    def parse_trans(item):
        
        chunk = item.split(' | ')
        
        if len(chunk) == 1:
            s_src = np.nan
            s_tgt = np.nan
            if chunk[0] == '':
                m_src = np.nan
                m_tgt = np.nan
            elif chunk[0][0] == '?':
                m_src = np.nan
                m_tgt = chunk[0].split(' -> ')[1]
            else:
                m_src = chunk[0].split(' -> ')[0]
                m_tgt = chunk[0].split(' -> ')[1]
        else:
            if chunk[1] == 'O':
                s_src = np.nan
                s_tgt = np.nan
            else:
                chunk1 = chunk[1].split(' -> ')
                if len(chunk1) == 1:
                    s_src = chunk1[0]
                    s_tgt = np.nan
                else:
                    s_src = chunk1[0] if chunk1[0] != 'O' else np.nan
                    s_tgt = chunk1[1] if chunk1[1] != 'O' else np.nan
                
            chunk1 = chunk[0].split(' -> ')
            if len(chunk1) == 1:
                m_src = chunk1[0]
                m_tgt = np.nan
            else:
                m_src = chunk1[0]
                m_tgt = chunk1[1]
                
        return m_src, m_tgt, s_src, s_tgt
    
    key_mapping = {
        'Conn_Rel': 'CXNR',
        'Conn_Req': 'CXNS',
        'LTE_HO': 'LTEH',
        'MN_HO': 'MCGH',
        'MN_HO_to_eNB': 'SCGR-II',
        'SN_setup': 'SCGA',
        'SN_Rel': 'SCGR-I',
        'SN_HO': 'SCGM',
        'RLF_II': 'MCGF',
        'RLF_III': 'NASR',
        'SCG_RLF': 'SCGF',
        'Add_SCell': 'SCLA'
    }
    
    D = parse_mi_ho(df, tz)
    
    # rename as acronym
    new_D = {key_mapping.get(key, key): value for key, value in D.items()}
    
    if df.empty:
        selected_cols = ['m_src1', 'm_tgt1', 's_src1', 's_tgt1', 'PCI', 'Cell Identity', 'eNB_ID', 'next_eNB_ID', 'Band ID', 'next_BID', 'DL frequency', 'UL frequency', 'DL bandwidth', 'UL bandwidth', 'TAC', 'MCC', 'MNC']
        table = pd.DataFrame(columns=['type', 'start', 'end', 'others', 'm_src', 'm_tgt', 's_src', 's_tgt', 'category', 'inter-eNB', 'inter-gNB', 'inter-Freq', 'band_cng', 'inter-RAT', '4G_5G', 'cause', 'near_before_RLF', *selected_cols])
        print('Empty RRC File!!!')
        return table, new_D
    
    table = pd.DataFrame()
    for key, lst in new_D.items():
        table1 = pd.DataFrame(lst, index=[key]*len(lst)).reset_index(names='type')
        table = pd.concat([table, table1], ignore_index=True)
    
    if table.empty:
        selected_cols = ['m_src1', 'm_tgt1', 's_src1', 's_tgt1', 'PCI', 'Cell Identity', 'eNB_ID', 'next_eNB_ID', 'Band ID', 'next_BID', 'DL frequency', 'UL frequency', 'DL bandwidth', 'UL bandwidth', 'TAC', 'MCC', 'MNC']
        table = pd.DataFrame(columns=['type', 'start', 'end', 'others', 'm_src', 'm_tgt', 's_src', 's_tgt', 'category', 'inter-eNB', 'inter-gNB', 'inter-Freq', 'band_cng', 'inter-RAT', '4G_5G', 'cause', 'near_before_RLF', *selected_cols])
        print('Handover Not Found!!!')
        return table, new_D
    
    # add Cell Identity & eNB ID
    sc_info = df[df['type_id'] == 'LTE_RRC_Serv_Cell_Info'][['Timestamp', 'type_id', 'PCI', 'Cell Identity', 'Band ID', 'DL frequency', 'UL frequency', 'DL bandwidth', 'UL bandwidth', 'TAC', 'MCC', 'MNC']].reset_index(drop=True).rename(columns={'Timestamp': 'start', 'type_id': 'type'})
    sc_info['eNB_ID'] = sc_info['Cell Identity'] // 256
    # sc_info['Cell_ID'] = sc_info['Cell Identity'] % 256
    sc_info = sc_info[['start', 'type', 'PCI', 'Cell Identity', 'eNB_ID', 'Band ID', 'DL frequency', 'UL frequency', 'DL bandwidth', 'UL bandwidth', 'TAC', 'MCC', 'MNC']]

    table = pd.concat([table, sc_info], ignore_index=True).sort_values(by='start').reset_index(drop=True)

    is_not_start = True
    selected_cols = ['PCI', 'Cell Identity', 'eNB_ID', 'Band ID', 'DL frequency', 'UL frequency', 'DL bandwidth', 'UL bandwidth', 'TAC', 'MCC', 'MNC']
    for i, row in table.iterrows():
        if row['type'] == 'LTE_RRC_Serv_Cell_Info':
            is_not_start = False
            info_to_fill = row[selected_cols].to_list()
            continue
        if is_not_start:
            continue
        table.loc[i, selected_cols] = info_to_fill

    table = table[table['type'] != 'LTE_RRC_Serv_Cell_Info'].reset_index(drop=True)
    
    # parse source & target cells
    for i, row in table.iterrows():
        table.loc[i, ['m_src', 'm_tgt', 's_src', 's_tgt']] = parse_trans(row['trans'])
    
    # distinguish intra/inter-eNB HO
    table1 = table[np.in1d(table['type'], ['SCLA', 'SCGA', 'SCGR-I', 'SCGF'])]
    table = table[~np.in1d(table['type'], ['SCLA', 'SCGA', 'SCGR-I', 'SCGF'])].reset_index(drop=True)
    
    table['next_eNB'] = table['eNB_ID'].shift(-1)
    for i, row in table.iloc[:-1].iterrows():
        if row['eNB_ID'] != row['next_eNB'] and row['type'] not in ['CXNS', 'CXNR']:
            if row['others'] == '':
                table.at[i, 'others'] = 'Inter eNB HO.'
            else:
                table.at[i, 'others'] += ' Inter eNB HO.'
    
    table = pd.concat([table, table1], ignore_index=True).sort_values(by='start').reset_index(drop=True)
    
    # label SCG Addition near after SCG Failure
    table1 = table[~np.in1d(table['type'], ['SCGA', 'SCGR-I', 'SCGR-II'])]
    table = table[np.in1d(table['type'], ['SCGA', 'SCGR-I', 'SCGR-II'])].reset_index(drop=True)
    
    table['prev_cmt'] = table['others'].shift(1)
    for i, row in table.iloc[1:].iterrows():
        if row['type'] == 'SCGA':
            if 'Near after SN_Rel' in row['others'] and 'Caused by scg-failure' in row['prev_cmt']:
                table.at[i, 'others'] += ' Caused by scg-failure.'
    
    # with pd.option_context('display.max_rows', None): 
    #     display(table)
    
    # combine closed SCG Addition & Release pair (which are not caused by scg-failure or RLF) into SCG Change
    table['next_end'] = table['end'].shift(-1)
    table['next_cmt'] = table['others'].shift(-1)
    table['next_s_tgt'] = table['s_tgt'].shift(-1)
    indices_to_remove = []
    for i, row in table.iloc[:-1].iterrows():
        if row['type'] == 'SCGR-I' and 'Near after SN_Rel' in row['next_cmt'] and row['s_src'] != row['next_s_tgt']:
            table.at[i, 'end'] = row['next_end']
            table.at[i, 's_tgt'] = row['next_s_tgt']
            table.at[i, 'type'] = 'SCGC-I'
            indices_to_remove.append(i+1)
        if row['type'] == 'SCGR-II' and 'Near after MN_HO_to_eNB' in row['next_cmt'] and row['s_src'] != row['next_s_tgt']:
            table.at[i, 'end'] = row['next_end']
            table.at[i, 's_tgt'] = row['next_s_tgt']
            table.at[i, 'type'] = 'SCGC-II'
            indices_to_remove.append(i+1)
    table = table.drop(indices_to_remove)
    
    # with pd.option_context('display.max_rows', None): 
    #     display(table)
    
    table = pd.concat([table, table1], ignore_index=True).sort_values(by='start').reset_index(drop=True)
    
    # re-classify eNB HO & MeNB HO
    table.loc[np.in1d(table['type'], ['LTEH']) & table['others'].str.contains('Inter eNB HO'), 'type'] = 'ENBH'
    table.loc[np.in1d(table['type'], ['MCGH']) & table['others'].str.contains('Inter eNB HO'), 'type'] = 'MNBH'
    
    # add the next eNB ID when meeting inter-eNB HO
    table1 = table[~table['others'].str.contains('Inter eNB HO')]
    table = table[table['others'].str.contains('Inter eNB HO')].reset_index(drop=True)
    
    table['next_eNB_ID'] = table['eNB_ID'].shift(-1)
    
    table = pd.concat([table, table1], ignore_index=True).sort_values(by='start').reset_index(drop=True)
    
    # detect band change and add the next Band ID when meeting inter-Freq HO
    band_mapping = {}
    for i, row in table[~table.duplicated(subset=['DL frequency'])].dropna(subset=['DL frequency']).iterrows():
        band_mapping[int(row['DL frequency'])] = row['Band ID']
    
    print(band_mapping)
    
    table1 = table[~table['others'].str.contains('Inter frequency HO')]
    table = table[table['others'].str.contains('Inter frequency HO')].reset_index(drop=True)
    
    table['next_BID'] = table['Band ID'].shift(-1)
    try:
        table.at[len(table)-1, 'next_BID'] = band_mapping[ast.literal_eval(table.iloc[-1]['m_tgt'])[1]]
    except:
        pass
    
    table = pd.concat([table, table1], ignore_index=True).sort_values(by='start').reset_index(drop=True)
    
    # check whether RLF is near after an HO event
    table1 = table[np.in1d(table['type'], ['CXNS', 'CXNR', 'SCLA'])]
    table = table[~np.in1d(table['type'], ['CXNS', 'CXNR', 'SCLA'])].reset_index(drop=True)
    
    table['next_start'] = table['start'].shift(-1)
    table['next_type'] = table['type'].shift(-1)
    table['near_before_RLF'] = False
    for i, row in table.iloc[:-1].iterrows():
        # check whether an HO event is near before an RLF (in 3 seconds)
        if row['type'] in ['LTEH', 'ENBH', 'MCGH', 'MNBH', 'SCGM', 'SCGA', 'SCGR-I', 'SCGR-II', 'SCGC-I', 'SCGC-II'] and \
            row['next_type'] in ['NASR', 'MCGF', 'SCGF']:
                if row['end'] > row['next_start'] - pd.Timedelta(seconds=1):
                    table.at[i, 'near_before_RLF'] = True
                    next_type = row['next_type']
                    if row['others'] == '':
                        table.at[i, 'others'] = f'Near before {next_type} 1 sec.'
                    else:
                        table.at[i, 'others'] += f' Near before {next_type} 1 sec.'
                        
                elif row['end'] > row['next_start'] - pd.Timedelta(seconds=2):
                    table.at[i, 'near_before_RLF'] = True
                    next_type = row['next_type']
                    if row['others'] == '':
                        table.at[i, 'others'] = f'Near before {next_type} 2 sec.'
                    else:
                        table.at[i, 'others'] += f' Near before {next_type} 2 sec.'
                        
                elif row['end'] > row['next_start'] - pd.Timedelta(seconds=3):
                    table.at[i, 'near_before_RLF'] = True
                    next_type = row['next_type']
                    if row['others'] == '':
                        table.at[i, 'others'] = f'Near before {next_type} 3 sec.'
                    else:
                        table.at[i, 'others'] += f' Near before {next_type} 3 sec.'
                    
    table = pd.concat([table, table1], ignore_index=True).sort_values(by='start').reset_index(drop=True)
    
    # label (PCI, BSID)
    table1 = table[np.in1d(table['type'], ['CXNS', 'CXNR', 'SCLA'])]
    table = table[~np.in1d(table['type'], ['CXNS', 'CXNR', 'SCLA'])].reset_index(drop=True)
    
    for col in ['m_src', 's_src']:
        new_col = f'{col}1'
        filtered_table = table[table[col].isna()].copy()
        filtered_table[new_col] = np.nan
        
        # add BSID (eNB)
        filtered_table1 = table[table[col].notna()].copy()
        filtered_table1[new_col] = filtered_table1['m_src'].astype(str) + ', ' + filtered_table1['eNB_ID'].astype(str)
        
        table = pd.concat([filtered_table, filtered_table1], ignore_index=True).sort_values(by='start').reset_index(drop=True)
    
    for col in ['m_tgt', 's_tgt']:
        new_col = f'{col}1'
        filtered_table = table[table[col].isna()].copy()
        filtered_table[new_col] = np.nan
        
        # add BSID (eNB)
        filtered_table1 = table[table[col].notna() & table['next_eNB_ID'].notna()].copy()
        filtered_table1[new_col] = filtered_table1['m_src'].astype(str) + ', ' + filtered_table1['next_eNB_ID'].astype(str)
        filtered_table2 = table[table[col].notna() & table['next_eNB_ID'].isna()].copy()
        filtered_table2[new_col] = filtered_table2['m_src'].astype(str) + ', ' + filtered_table2['eNB_ID'].astype(str)
        
        table = pd.concat([filtered_table, filtered_table1, filtered_table2], ignore_index=True).sort_values(by='start').reset_index(drop=True)
    
    table = pd.concat([table, table1], ignore_index=True).sort_values(by='start').reset_index(drop=True)
    
    # add category
    table['category'] = 'Others'
    table.loc[np.in1d(table['type'], ['LTEH', 'ENBH', 'MCGH', 'MNBH', 'SCGM', 'SCGA', 'SCGR-I', 'SCGC-I', 'SCGR-II', 'SCGC-II']), 'category'] = 'HO'
    table.loc[np.in1d(table['type'], ['MCGF', 'NASR', 'SCGF']), 'category'] = 'RLF'

    # add failure cause
    failure_cause = [
        'reconfigurationFailure (0)', 'handoverFailure (1)', 'otherFailure (2)',
        't310-Expiry (0)', 'randomAccessProblem (1)', 'rlc-MaxNumRetx (2)', 'synchReconfigFailureSCG (3)', 'scg-ReconfigFailure (4)', 'srb3-IntegrityFailure (5)', 'other-r16 (6)'
    ]
    
    for tag in failure_cause:
        table.loc[table['others'].str.contains(tag, regex=False), 'cause'] = tag
        table['others'] = table['others'].str.replace(f" {tag}.", "", regex=False)
        table['others'] = table['others'].str.replace(f"{tag}.", "", regex=False)
    
    # add Access Technology type
    table['4G_5G'] = '4G'
    table.loc[np.in1d(table['type'], ['SCGM', 'SCGA', 'SCGR-I', 'SCGC-I', 'SCGF']), '4G_5G'] = '5G'
    table.loc[np.in1d(table['type'], ['SCGR-II', 'SCGC-II']), '4G_5G'] = '4G_5G'
    
    # add more boolean columns
    table['inter-eNB'] = False
    table.loc[table['others'].str.contains('Inter eNB HO'), 'inter-eNB'] = True
    table['others'] = table['others'].str.replace(" Inter eNB HO.", "")
    table['others'] = table['others'].str.replace("Inter eNB HO.", "")
    
    table['inter-Freq'] = False
    table.loc[table['others'].str.contains('Inter frequency HO'), 'inter-Freq'] = True
    table['others'] = table['others'].str.replace(" Inter frequency HO.", "")
    table['others'] = table['others'].str.replace("Inter frequency HO.", "")
    
    table['band_cng'] = False
    table.loc[table['inter-Freq'] & (table['Band ID'] != table['next_BID']), 'band_cng'] = True
    
    table['inter-RAT'] = False
    table.loc[np.in1d(table['type'], ['SCGA', 'SCGR-I', 'SCGC-I', 'SCGR-II', 'SCGC-II']), 'inter-RAT'] = True
    
    table['inter-gNB'] = False
    table.loc[np.in1d(table['type'], ['SCGC-I', 'SCGC-II']), 'inter-gNB'] = True
    
    # find row na-"end" & fill with "start"
    if endfill:
        nan_end_rows = table[table['end'].isnull()]
        table.loc[nan_end_rows.index, 'end'] = nan_end_rows['start']
    
    # ignore CXNS, CXNR, SCLA
    table = table[~np.in1d(table['type'], ['CXNS', 'CXNR', 'SCLA'])].reset_index(drop=True)
    
    # remove SCG Addition, Release caused by SCG Failure or any other RLFs if needed (default: True)
    if radical:
        table = table[~((table['others'].str.contains('Caused by scg-failure') | table['others'].str.contains('Near after RLF')))].reset_index(drop=True)
    
    # select columns
    selected_cols = ['m_src1', 'm_tgt1', 's_src1', 's_tgt1', 'PCI', 'Cell Identity', 'eNB_ID', 'next_eNB_ID', 'Band ID', 'next_BID', 'DL frequency', 'UL frequency', 'DL bandwidth', 'UL bandwidth', 'TAC', 'MCC', 'MNC']
    table = table[['type', 'start', 'end', 'others', 'm_src', 'm_tgt', 's_src', 's_tgt', 'category', 'inter-eNB', 'inter-gNB', 'inter-Freq', 'band_cng', 'inter-RAT', '4G_5G', 'cause', 'near_before_RLF', *selected_cols]]
    
    return table, new_D

# Input Dataset

In [41]:
import os
import json

# Specify root directory
root_dir = "/Users/jackbedford/Desktop/MOXA/Code/data"

# Collect dates
dates = [s for s in sorted(os.listdir(root_dir)) if os.path.isdir(os.path.join(root_dir, s)) and s != 'backup']
date_dirs = [os.path.join(root_dir, s) for s in dates]

for date, date_dir in zip(dates, date_dirs):
    # Specify the path to your JSON file
    date = os.path.basename(date_dir)
    json_filepath = os.path.join(date_dir, f'{date}.json')
    
    # Read the JSON file and load its contents into a dictionary
    with open(json_filepath, 'r', encoding='utf-8') as json_file:
        my_dict = json.load(json_file)
    
    if not my_dict:
        continue
        
    print(date, len(my_dict))
    
    for expr, item in my_dict.items():
        print({expr: item})

2023-03-16 1
{'UDP_Bandlock_B1_B7_B8_RM500Q': {'route': 'BR', 'devices': {'qc00': 'B1', 'qc02': 'B7', 'qc03': 'B8'}, 'ods': [[0, 'Origin Station', 'Destination Station', 'Departure Time', 'Notes'], [1, '萬芳醫院', '動物園', '19:23', ''], [2, '動物園', '萬芳醫院', '19:32', ''], [3, '萬芳醫院', '動物園', '19:39', ''], [4, '動物園', '萬芳醫院', '19:47', ''], [5, '萬芳醫院', '動物園', '19:55', ''], [6, '動物園', '萬芳醫院', '20:03', ''], [7, '萬芳醫院', '動物園', '20:09', ''], [8, '動物園', '萬芳醫院', '20:18', ''], [9, '萬芳醫院', '動物園', '20:27', ''], [10, '動物園', '萬芳醫院', '20:33', ''], [11, '萬芳醫院', '動物園', '20:42', ''], [12, '動物園', '萬芳醫院', '20:49', '']], 'stations': ['動物園', '木柵', '萬芳社區', '萬芳醫院'], 'telecom': 'CHT', 'protocol': 'UDP', 'length': [250, 'byte'], 'bitrate': [1, 'Mbps'], 'notes': ''}}
2023-03-26 1
{'UDP_Bandlock_All_RM500Q': {'route': 'BR', 'devices': {'qc00': 'All', 'qc02': 'All', 'qc03': 'All'}, 'ods': [[0, 'Origin Station', 'Destination Station', 'Departure Time', 'Notes'], [1, '萬芳醫院', '動物園', '16:36', ''], [2, '動物園', '萬芳醫院', '16:46', ''

In [42]:
# Select Dates
# sel_dates = [s for s in dates if s >= '2023-09-12']
# sel_dates = [s for s in dates if s < '2023-09-12']
# sel_dates = [s for s in dates if s >= '2023-08-16' and s < '2023-09-12']
# sel_dates = dates[:5]
# sel_dates = dates[-8:]
sel_dates = ['2023-09-12-2', '2023-09-21', '2023-11-09', '2023-10-05', '2023-10-19', '2023-10-26', '2023-11-01', '2023-11-02']
# sel_dates = dates[:]
# exc_dates = ['2023-06-12', '2023-06-13']
exc_dates = []
date_dirs = [os.path.join(root_dir, s) for s in sel_dates if s not in exc_dates]

# Select Experiment Names
# sel_exps = ['UDP_Bandlock_9S_Phone_Brown', 'UDP_Bandlock_9S_Phone_Airport']
sel_exps = []
exc_exps = ['Modem_Action_Test', 'Control_Group', 'Control_Group2', 'Control_Group3']
# exc_exps = []

# Select Routes
# sel_routes = ['BR']
sel_routes = []

In [43]:
# Collect Experiments
exps_dict = {}

for date_dir in date_dirs:
    date = os.path.basename(date_dir)
    
    # Specify the path to your JSON file
    json_filepath = os.path.join(date_dir, f'{date}.json')
    
    # Read the JSON file and load its contents into a dictionary
    with open(json_filepath, 'r', encoding='utf-8') as json_file:
        my_dict = json.load(json_file)
    
    if not my_dict:
        continue
    
    for i, (expr, item) in enumerate(my_dict.items()):
        if len(sel_exps) != 0 and expr not in sel_exps:
            continue
        
        if len(exc_exps) != 0 and expr in exc_exps:
            continue
        
        if len(sel_routes) != 0 and item['route'] not in sel_routes:
            continue
        
        try:
            exps_dict[date] = {**exps_dict[date], **{expr: item}}
        except:
            exps_dict[date] = {expr: item}

for date, exps in exps_dict.items():
    # print(date, len(exps), exps)
    print(date, len(exps))
    
    for expr, item in exps.items():
        print({expr: item})

2023-09-12-2 1
{'UDP_Bandlock_9S_Phone_Brown': {'route': 'BR', 'devices': {'sm00': 'All', 'sm01': 'All', 'sm02': 'B3', 'sm03': 'B7', 'sm04': 'B8', 'sm05': 'B3B7', 'sm06': 'B3B8', 'sm07': 'B7B8', 'sm08': 'LTE'}, 'ods': [[0, 'Origin Station', 'Destination Station', 'Departure Time', 'Notes'], [1, '動物園', '南港展覽館', '13:34', '到木柵才想起要開GPS'], [2, '南港展覽館', '動物園', '14:52', '']], 'stations': ['動物園', '木柵', '萬芳社區', '萬芳醫院', '辛亥', '麟光', '六張犁', '科技大樓', '大安', '忠孝復興', '南京復興', '中山國中', '松山機場', '大直', '劍南路', '西湖', '港墘', '文德', '內湖', '大湖公園', '葫洲', '東湖', '南港軟體園區', '南港展覽館'], 'telecom': 'CHT', 'protocol': 'UDP', 'length': [250, 'byte'], 'bitrate': [1, 'Mbps'], 'notes': ''}}
2023-09-21 1
{'UDP_Bandlock_9S_Phone_Brown': {'route': 'BR', 'devices': {'sm00': 'All', 'sm01': 'All', 'sm02': 'B3', 'sm03': 'B7', 'sm04': 'B8', 'sm05': 'B3B7', 'sm06': 'B3B8', 'sm07': 'B7B8', 'sm08': 'LTE'}, 'ods': [[0, 'Origin Station', 'Destination Station', 'Departure Time', 'Notes'], [1, '動物園', '南港展覽館', '14:29', ''], [2, '南港展覽館', '動物園', 

In [49]:
root_dir = '/Users/jackbedford/Desktop/MOXA/Code/data'
dates = ['2023-09-12-2', '2023-09-21', '2023-11-09', '2023-10-05', '2023-10-19', '2023-10-26', '2023-11-01', '2023-11-02']
routes = ['BR', 'BR', 'BR', 'G', 'R', 'A', 'A', 'A']
expr_name = [list(s.keys())[0] for s in exps_dict.values()]
devices = ['sm00', 'sm01']
traces = ['#01', '#02']

rrc_files = {route: [] for route in routes}
dl_files = {route: [] for route in routes}
ul_files = {route: [] for route in routes}
nr_files = {route: [] for route in routes}
lte_files = {route: [] for route in routes}
ho_files = {route: [] for route in routes}

for date, route, name in zip(dates, routes, expr_name):
    for dev in devices:
        for tr in traces:
            data_dir = os.path.join(root_dir, date, name, dev, tr, 'data')
            rrc_files[route] += [os.path.join(data_dir, s) for s in os.listdir(data_dir) if s.endswith('_rrc.csv')]
            dl_files[route] += [os.path.join(data_dir, s) for s in os.listdir(data_dir) if s.endswith('udp_dnlk_loss_latency.csv')]
            ul_files[route] += [os.path.join(data_dir, s) for s in os.listdir(data_dir) if s.endswith('udp_uplk_loss_latency.csv')]
            nr_files[route] += [os.path.join(data_dir, s) for s in os.listdir(data_dir) if s.endswith('_nr_ml1.csv')]
            lte_files[route] += [os.path.join(data_dir, s) for s in os.listdir(data_dir) if s.endswith('ml1.csv') and not s.endswith('nr_ml1.csv')]
            ho_files[route] += [os.path.join(data_dir, s) for s in os.listdir(data_dir) if s.endswith('ho_related_info.csv')]

In [50]:
ho_files

{'BR': ['/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12-2/UDP_Bandlock_9S_Phone_Brown/sm00/#01/data/ho_related_info.csv',
  '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12-2/UDP_Bandlock_9S_Phone_Brown/sm00/#02/data/ho_related_info.csv',
  '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12-2/UDP_Bandlock_9S_Phone_Brown/sm01/#01/data/ho_related_info.csv',
  '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12-2/UDP_Bandlock_9S_Phone_Brown/sm01/#02/data/ho_related_info.csv',
  '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-21/UDP_Bandlock_9S_Phone_Brown/sm00/#01/data/ho_related_info.csv',
  '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-21/UDP_Bandlock_9S_Phone_Brown/sm00/#02/data/ho_related_info.csv',
  '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-21/UDP_Bandlock_9S_Phone_Brown/sm01/#01/data/ho_related_info.csv',
  '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-21/UDP_Bandlock_9S_Phone_Brown/sm01/#02/data/ho_related_info.csv',
  '/Users/jackbedford/Desk

In [51]:
import pickle

# Specify the file path where you want to save the pickle file
file_path = "./temp/2023-11-22_dl_expr_duras.pkl"

if os.path.isfile(file_path):
    # Load the dictionary from the pickle file
    with open(file_path, 'rb') as f:
        expr_duras = pickle.load(f)
else:
    expr_duras = {route: [] for route in routes}
    
    for key in dl_files.keys():
        for dl_file in dl_files[key]:
            df_dl = set_data(pd.read_csv(dl_file))
            start_time = df_dl.iloc[0]['arr_time']
            end_time = df_dl.iloc[-1]['arr_time']
            dura = (end_time - start_time).total_seconds()
            expr_duras[key].append((start_time, end_time, dura))

    # Save the dictionary to a pickle file
    with open(file_path, 'wb') as f:
        pickle.dump(expr_duras, f)

# pprint(expr_duras)

# Stage 1: 請統計並比較各種event發生的頻率。不同的路線上的發生比率是否類似?

In [52]:
table_ho = pd.DataFrame(columns=['Route', 'HO', 'HO_freq', 'LTEH', 'ENBH', 'MCGH', 'MNBH', 'SCGM', 'Intra-RAT', 'SCGA', 'SCGR-I', 'SCGC-I', 'SCGR-II', 'SCGC-II', 'Inter-RAT', 'Dura'])
table_rlf = pd.DataFrame(columns=['Route', 'RLF', 'RLF_freq', 'MCGF', 'NASR', 'SCGF', 'Dura'])
table_mcgf = pd.DataFrame(columns=['Route', 'MCGF', 'MCGF_freq', 'reconfigurationFailure (0)', 'handoverFailure (1)', 'otherFailure (2)', 'Dura'])
table_nasr = pd.DataFrame(columns=['Route', 'NASR', 'NASR_freq', 'reconfigurationFailure (0)', 'handoverFailure (1)', 'otherFailure (2)', 'Dura'])
table_scgf = pd.DataFrame(columns=['Route', 'SCGF', 'SCGF_freq', 't310-Expiry (0)', 'randomAccessProblem (1)', 'rlc-MaxNumRetx (2)', 'synchReconfigFailureSCG (3)', 'scg-ReconfigFailure (4)', 'srb3-IntegrityFailure (5)', 'other-r16 (6)', 'Dura'])

In [53]:
for key in ho_files.keys():
    for i, (ho_file, nr_file, dura) in enumerate(zip(ho_files[key], nr_files[key], expr_duras[key])):
        print((i, ho_file, nr_file))
        
        df_nr = pd.read_csv(nr_file)
        if df_nr.empty:
            print('Never detect 5G-NSA signal!!!')
            display(df_nr)
            continue
        
        df_ho = pd.read_csv(ho_file)
        df_ho['start'] = pd.to_datetime(df_ho['start'])
        df_ho['end'] = pd.to_datetime(df_ho['end'])
        
        start_time = dura[0]
        try:
            end_time = min(dura[1], df_ho[df_ho['cause'] == 'rlc-MaxNumRetx (2)'].iloc[0]['start'])
        except:
            print('Great!! No rlc-MaxNumRetx (2) occurs!!')
            end_time = dura[1]
            
        _dura = (end_time - start_time).total_seconds()
        df_ho = df_ho[(df_ho['start'] >= start_time) & (df_ho['end'] < end_time)].reset_index(drop=True)
        
        new_row_ho = pd.DataFrame([[
            key,
            sum(df_ho['category'] == 'HO'),
            round(sum(df_ho['category'] == 'HO') / dura[2] * 60 * 60, 2),
            sum(df_ho['type'] == 'LTEH'),
            sum(df_ho['type'] == 'ENBH'),
            sum(df_ho['type'] == 'MCGH'),
            sum(df_ho['type'] == 'MNBH'),
            sum(df_ho['type'] == 'SCGM'),
            sum((df_ho['category'] == 'HO') & ~df_ho['inter-RAT']),
            sum(df_ho['type'] == 'SCGA'),
            sum(df_ho['type'] == 'SCGR-I'),
            sum(df_ho['type'] == 'SCGC-I'),
            sum(df_ho['type'] == 'SCGR-II'),
            sum(df_ho['type'] == 'SCGC-II'),
            sum((df_ho['category'] == 'HO') & df_ho['inter-RAT']),
            round(_dura, 3)
        ]], columns=table_ho.columns)

        # display(new_row_ho)
        table_ho = table_ho.append(new_row_ho, ignore_index=True)

        new_row_rlf = pd.DataFrame([[
            key,
            sum(df_ho['category'] == 'RLF'),
            round(sum(df_ho['category'] == 'RLF') / dura[2] * 60 * 60, 2),
            sum(df_ho['type'] == 'MCGF'),
            sum(df_ho['type'] == 'NASR'),
            sum(df_ho['type'] == 'SCGF'),
            round(_dura, 3)
        ]], columns=table_rlf.columns)

        # display(new_row_rlf)
        table_rlf = table_rlf.append(new_row_rlf, ignore_index=True)

        new_row_mcgf = pd.DataFrame([[
            key,
            sum(df_ho['type'] == 'MCGF'),
            round(sum(df_ho['type'] == 'MCGF') / dura[2] * 60 * 60, 2),
            sum((df_ho['type'] == 'MCGF') & (df_ho['cause'] == 'reconfigurationFailure (0)')),
            sum((df_ho['type'] == 'MCGF') & (df_ho['cause'] == 'handoverFailure (1)')),
            sum((df_ho['type'] == 'MCGF') & (df_ho['cause'] == 'otherFailure (2)')),
            round(_dura, 3)
        ]], columns=table_mcgf.columns)

        # display(new_row_mcgf)
        table_mcgf = table_mcgf.append(new_row_mcgf, ignore_index=True)

        new_row_nasr = pd.DataFrame([[
            key,
            sum(df_ho['type'] == 'NASR'),
            round(sum(df_ho['type'] == 'NASR') / dura[2] * 60 * 60, 2),
            sum((df_ho['type'] == 'NASR') & (df_ho['cause'] == 'reconfigurationFailure (0)')),
            sum((df_ho['type'] == 'NASR') & (df_ho['cause'] == 'handoverFailure (1)')),
            sum((df_ho['type'] == 'NASR') & (df_ho['cause'] == 'otherFailure (2)')),
            round(_dura, 3)
        ]], columns=table_nasr.columns)

        # display(new_row_nasr)
        table_nasr = table_nasr.append(new_row_nasr, ignore_index=True)

        new_row_scgf = pd.DataFrame([[
            key,
            sum(df_ho['type'] == 'SCGF'),
            round(sum(df_ho['type'] == 'SCGF') / dura[2] * 60 * 60, 2),
            sum((df_ho['type'] == 'SCGF') & (df_ho['cause'] == 't310-Expiry (0)')),
            sum((df_ho['type'] == 'SCGF') & (df_ho['cause'] == 'randomAccessProblem (1)')),
            sum((df_ho['type'] == 'SCGF') & (df_ho['cause'] == 'rlc-MaxNumRetx (2)')),
            sum((df_ho['type'] == 'SCGF') & (df_ho['cause'] == 'synchReconfigFailureSCG (3)')),
            sum((df_ho['type'] == 'SCGF') & (df_ho['cause'] == 'scg-ReconfigFailure (4)')),
            sum((df_ho['type'] == 'SCGF') & (df_ho['cause'] == 'srb3-IntegrityFailure (5)')),
            sum((df_ho['type'] == 'SCGF') & (df_ho['cause'] == 'other-r16 (6)')),
            round(_dura, 3)
        ]], columns=table_scgf.columns)

        # display(new_row_scgf)
        table_scgf = table_scgf.append(new_row_scgf, ignore_index=True)
        
    #     break
    # break

file_paths = [
    "./temp/2023-11-22_frequency_table_ho.pkl",
    "./temp/2023-11-22_frequency_table_rlf.pkl",
    "./temp/2023-11-22_frequency_table_mcgf.pkl",
    "./temp/2023-11-22_frequency_table_nasr.pkl",
    "./temp/2023-11-22_frequency_table_scgf.pkl"
]

for file_path, table in zip(file_paths, [table_ho, table_rlf, table_mcgf, table_nasr, table_scgf]):
    # Save the dictionary to a pickle file
    with open(file_path, 'wb') as f:
        # print(file_path)
        # display(table)
        pickle.dump(table, f)

(0, '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12-2/UDP_Bandlock_9S_Phone_Brown/sm00/#01/data/ho_related_info.csv', '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12-2/UDP_Bandlock_9S_Phone_Brown/sm00/#01/data/diag_log_sm00_2023-09-12_13-34-15_nr_ml1.csv')
Great!! No rlc-MaxNumRetx (2) occurs!!
(1, '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12-2/UDP_Bandlock_9S_Phone_Brown/sm00/#02/data/ho_related_info.csv', '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12-2/UDP_Bandlock_9S_Phone_Brown/sm00/#02/data/diag_log_sm00_2023-09-12_14-52-28_nr_ml1.csv')
Great!! No rlc-MaxNumRetx (2) occurs!!
(2, '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12-2/UDP_Bandlock_9S_Phone_Brown/sm01/#01/data/ho_related_info.csv', '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12-2/UDP_Bandlock_9S_Phone_Brown/sm01/#01/data/diag_log_sm01_2023-09-12_13-34-15_nr_ml1.csv')
Great!! No rlc-MaxNumRetx (2) occurs!!
(3, '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12-2/UDP_Bandlock_9S_

,Timestamp,type_id,Raster ARFCN,Num Cells,Serving Cell Index,Serving Cell PCI


(6, '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-21/UDP_Bandlock_9S_Phone_Brown/sm01/#01/data/ho_related_info.csv', '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-21/UDP_Bandlock_9S_Phone_Brown/sm01/#01/data/diag_log_sm01_2023-09-21_14-29-12_nr_ml1.csv')
Great!! No rlc-MaxNumRetx (2) occurs!!
(7, '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-21/UDP_Bandlock_9S_Phone_Brown/sm01/#02/data/ho_related_info.csv', '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-21/UDP_Bandlock_9S_Phone_Brown/sm01/#02/data/diag_log_sm01_2023-09-21_15-28-46_nr_ml1.csv')
Never detect 5G-NSA signal!!!


,Timestamp,type_id,Raster ARFCN,Num Cells,Serving Cell Index,Serving Cell PCI


(8, '/Users/jackbedford/Desktop/MOXA/Code/data/2023-11-09/UDP_Bandlock_9S_Phone_Brown/sm00/#01/data/ho_related_info.csv', '/Users/jackbedford/Desktop/MOXA/Code/data/2023-11-09/UDP_Bandlock_9S_Phone_Brown/sm00/#01/data/diag_log_sm00_2023-11-09_12-20-44_nr_ml1.csv')
(9, '/Users/jackbedford/Desktop/MOXA/Code/data/2023-11-09/UDP_Bandlock_9S_Phone_Brown/sm00/#02/data/ho_related_info.csv', '/Users/jackbedford/Desktop/MOXA/Code/data/2023-11-09/UDP_Bandlock_9S_Phone_Brown/sm00/#02/data/diag_log_sm00_2023-11-09_13-40-46_nr_ml1.csv')
(10, '/Users/jackbedford/Desktop/MOXA/Code/data/2023-11-09/UDP_Bandlock_9S_Phone_Brown/sm01/#01/data/ho_related_info.csv', '/Users/jackbedford/Desktop/MOXA/Code/data/2023-11-09/UDP_Bandlock_9S_Phone_Brown/sm01/#01/data/diag_log_sm01_2023-11-09_12-20-44_nr_ml1.csv')
(11, '/Users/jackbedford/Desktop/MOXA/Code/data/2023-11-09/UDP_Bandlock_9S_Phone_Brown/sm01/#02/data/ho_related_info.csv', '/Users/jackbedford/Desktop/MOXA/Code/data/2023-11-09/UDP_Bandlock_9S_Phone_Brown

In [54]:
tmp_tables = []
file_paths = [
    "./temp/2023-11-22_frequency_table_ho.pkl",
    "./temp/2023-11-22_frequency_table_rlf.pkl",
    "./temp/2023-11-22_frequency_table_mcgf.pkl",
    "./temp/2023-11-22_frequency_table_nasr.pkl",
    "./temp/2023-11-22_frequency_table_scgf.pkl"
]

for file_path in file_paths:
    # Load the dictionary from the pickle file
    with open(file_path, 'rb') as f:
        tmp_table = pickle.load(f)
        # print(file_path)
        # display(tmp_table)
        tmp_tables.append(tmp_table)

table_ho = tmp_tables[0]
table_rlf = tmp_tables[1]
table_mcgf = tmp_tables[2]
table_nasr = tmp_tables[3]
table_scgf = tmp_tables[4]

In [55]:
display(table_ho)
display(table_rlf)
display(table_mcgf)
display(table_nasr)
display(table_scgf)

,Route,HO,HO_freq,LTEH,ENBH,MCGH,MNBH,SCGM,Intra-RAT,SCGA,SCGR-I,SCGC-I,SCGR-II,SCGC-II,Inter-RAT,Dura
0,BR,1,1.27,1,0,0,0,0,1,0,0,0,0,0,0,2835.088
1,BR,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,2898.128
2,BR,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,2835.102
3,BR,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,2897.504
4,BR,194,243.18,45,149,0,0,0,194,0,0,0,0,0,0,2871.952
5,BR,193,241.93,43,150,0,0,0,193,0,0,0,0,0,0,2871.857
6,BR,148,192.30,1,0,20,49,70,140,3,1,0,2,2,8,1466.645
7,BR,164,206.02,2,1,18,51,75,147,5,2,0,4,6,17,1278.888
8,BR,127,165.01,1,1,16,47,55,120,3,0,1,3,0,7,1250.240
9,BR,152,190.96,2,0,20,57,62,141,2,1,1,1,6,11,1104.622


,Route,RLF,RLF_freq,MCGF,NASR,SCGF,Dura
0,BR,0,0.00,0,0,0,2835.088
1,BR,0,0.00,0,0,0,2898.128
2,BR,0,0.00,0,0,0,2835.102
3,BR,0,0.00,0,0,0,2897.504
4,BR,6,7.52,6,0,0,2871.952
5,BR,7,8.77,7,0,0,2871.857
6,BR,6,7.80,6,0,0,1466.645
7,BR,16,20.10,12,0,4,1278.888
8,BR,9,11.69,8,1,0,1250.240
9,BR,13,16.33,12,0,1,1104.622


,Route,MCGF,MCGF_freq,reconfigurationFailure (0),handoverFailure (1),otherFailure (2),Dura
0,BR,0,0.00,0,0,0,2835.088
1,BR,0,0.00,0,0,0,2898.128
2,BR,0,0.00,0,0,0,2835.102
3,BR,0,0.00,0,0,0,2897.504
4,BR,6,7.52,0,0,6,2871.952
5,BR,7,8.77,0,0,7,2871.857
6,BR,6,7.80,0,0,6,1466.645
7,BR,12,15.07,0,0,12,1278.888
8,BR,8,10.39,0,0,8,1250.240
9,BR,12,15.08,0,1,11,1104.622


,Route,NASR,NASR_freq,reconfigurationFailure (0),handoverFailure (1),otherFailure (2),Dura
0,BR,0,0.00,0,0,0,2835.088
1,BR,0,0.00,0,0,0,2898.128
2,BR,0,0.00,0,0,0,2835.102
3,BR,0,0.00,0,0,0,2897.504
4,BR,0,0.00,0,0,0,2871.952
5,BR,0,0.00,0,0,0,2871.857
6,BR,0,0.00,0,0,0,1466.645
7,BR,0,0.00,0,0,0,1278.888
8,BR,1,1.30,0,0,1,1250.240
9,BR,0,0.00,0,0,0,1104.622


,Route,SCGF,SCGF_freq,t310-Expiry (0),randomAccessProblem (1),rlc-MaxNumRetx (2),synchReconfigFailureSCG (3),scg-ReconfigFailure (4),srb3-IntegrityFailure (5),other-r16 (6),Dura
0,BR,0,0.00,0,0,0,0,0,0,0,2835.088
1,BR,0,0.00,0,0,0,0,0,0,0,2898.128
2,BR,0,0.00,0,0,0,0,0,0,0,2835.102
3,BR,0,0.00,0,0,0,0,0,0,0,2897.504
4,BR,0,0.00,0,0,0,0,0,0,0,2871.952
5,BR,0,0.00,0,0,0,0,0,0,0,2871.857
6,BR,0,0.00,0,0,0,0,0,0,0,1466.645
7,BR,4,5.02,0,4,0,0,0,0,0,1278.888
8,BR,0,0.00,0,0,0,0,0,0,0,1250.240
9,BR,1,1.26,0,0,0,1,0,0,0,1104.622


In [56]:
root = '/Users/jackbedford/Desktop/MOXA/Code/results/2023-10-31 results'
file_paths = [
    "./2023-11-22_frequency_table_ho.csv",
    "./2023-11-22_frequency_table_rlf.csv",
    "./2023-11-22_frequency_table_mcgf.csv",
    "./2023-11-22_frequency_table_nasr.csv",
    "./2023-11-22_frequency_table_scgf.csv"
]

for file_path, table in zip(file_paths, [table_ho, table_rlf, table_mcgf, table_nasr, table_scgf]):
    # Save the dictionary to a pickle file
    table.to_csv(os.path.join(root, file_path), index=False)